In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
import torch
from torch import nn
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize, RegexpTokenizer
import re
from pathlib import Path
from torchtext.vocab import FastText, vocab
import json
import string

from srcs.dataset import TextDataset
from srcs.embedding import get_fasttext_vectors
from transformers import BertTokenizer

In [96]:
DATASET_PATH = 'datasets/dataset.csv'

In [97]:
def load_data():
    return pd.read_csv(DATASET_PATH, sep='|', index_col=0)

In [98]:
df = load_data()

In [17]:
#nltk.download('stopwords')

In [2]:
french_stopwords = stopwords.words('french')

In [3]:
def remove_stopwords(text):
    return [token for token in text if token.lower() not in french_stopwords]

In [4]:
def tokenize(text_data):
    return sent_tokenize(text_data, language='french')

In [8]:
device = torch.device('cpu')

In [127]:
tokenizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
def preprocess(text):
    return text

def tokenize_sentence(sentence: str,tokenizer: RegexpTokenizer):
    '''Simple tokenizer, removes or replaces special characters
    sentence : str sentence to be tokenized
    tokenizer : tokenizer with tokenize method '''

    #Lower capital leters
    tokenized=sentence.lower()
    #Change special character
    tokenized=re.sub("’","'",tokenized)
    #Remove unwanted characters
    tokenized=re.sub("(@\w*\\b\s?|#\w*\\b\s?|&\w*\\b\s?|\n\s?|\\\\|\<|\>|\||\*)","",tokenized)
    tokenized=re.sub("\/","",tokenized)
    #Replace articles since model does not embed contractions well
    tokenized=re.sub("l'","le ",tokenized)
    tokenized=re.sub("d'","de ",tokenized)
    tokenized=re.sub("j'","je ",tokenized)
    tokenized=re.sub("qu'","que ",tokenized)
    tokenized=re.sub("t'","te ",tokenized)
    tokenized=re.sub("c'","ce ",tokenized)
    tokenized = tokenized.lower() 
    tokenized=tokenized.strip()  
    tokenized=re.compile('<.*?>').sub('', tokenized) 
    tokenized = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', tokenized)  
    tokenized = re.sub('\s+', ' ', tokenized)  
    tokenized = re.sub(r'\[[0-9]*\]',' ',tokenized) 
    tokenized=re.sub(r'[^\w\s]', '', str(tokenized).lower().strip())
    tokenized = re.sub(r'\d',' ',tokenized) 
    tokenized = re.sub(r'\s+',' ',tokenized) 
      #Tokenize sentence
    tokenized=tokenizer.tokenize(tokenized)
    return(tokenized)
tokenize = lambda text : tokenize_sentence(text, tokenizer)

In [130]:
v
def pad(token_sequence, sequence_size):
    '''Function to pad sequence
    token_sequence : list of tokens of len < self.sequence_size'''
    pad_token="<pad>"
    start_token="<s>"
    end_token="</s>"
    pad_size = sequence_size-len(token_sequence)
    result = [start_token] + token_sequence + [pad_token for i in range(pad_size)] + [end_token]
    return result

def sample(tokens):
    ''' sample sequence from token sequence
    tokens : list of tokens'''
    pad_token="<pad>",
    start_token="<s>",
    end_token="</s>"

    nb_tokens=len(tokens)
    starting_index=np.random.randint(nb_tokens)
    if starting_index + self.sequence_size < nb_tokens :
      result = [self.start_token] + tokens[starting_index : starting_index + self.sequence_size] + [self.end_token]
    else:
      result= tokens[starting_index : nb_tokens]
      result=self.pad(result)
    return result

def process(tokens):
    sequence=[vocab_stoi[token] for token in sample(tokens)]
    return (sequence,label)

In [125]:
df['tokenized'] = df.text.apply(str).apply(tokenize)

In [116]:
ft_vectors, vocab_stoi = get_fasttext_vectors()
embedder = nn.Embedding.from_pretrained(ft_vectors.vectors, freeze=True)

 12%|████████▋                                                               | 43235/356898 [00:02<00:20, 15538.14it/s]


KeyboardInterrupt: 

In [ ]:
sequence=[self.stoi[token] for token in self.sample(tokens)]
    return(sequence,label)

embedded = embedder.embed()

In [126]:
df

,text,label,tokenized
0,"Les ""grands dangers"" qui guettent les voyageur...",0,les grands dangers qui guettent les voyageurs ...
1,\nOn prête au premier ministre de l'État d'I...,0,on prête au premier ministre de le état de isr...
2,L e cortège prend des allures de visite guidée...,0,l e cortège prend des allures de visite guidée...
3,U ne pluie de graines et d’informations s’est ...,0,u ne pluie de graines et de informations s est...
4,Fiscalité du patrimoine : ce que proposent les...,0,fiscalité du patrimoine ce que proposent les c...
...,...,...,...
243,"Je pourrais, bien que l'erreur soit plus grave...",1,je pourrais bien que le erreur soit plus grave...
244,"Je me disais aussi : "" Non seulement est-il en...",1,je me disais aussi non seulement est il encore...
245,J'éprouvais un sentiment de fatigue profonde à...,1,je éprouvais un sentiment de fatigue profonde ...
246,La date à laquelle j'entendais le bruit de la ...,1,la date à laquelle je entendais le bruit de la...


In [131]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
def return_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]